## Introduction

Hello, my readers, I'm Parthiv. Quick note: **This file will have a lot of playing around and searching for trends!** The `dist/` directory will have a lot of the final graphs and analysis, so make sure to explore that after. I'll try to explain what I'm doing as we go on, but I apologize if anything's unclear.

In [3]:
# Importing necessary libraries
# pip install pandas numpy matplotlib seaborn plotly scienceplots

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


### Config for Matplotlib, Seaborn, Plotly, LaTeX

Style config. Prioritizing aesthetics but also should be scientifically appropriate.

In [5]:
# Set global figure size
plt.rcParams['figure.figsize'] = [10, 6]

# Set font sizes
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16

plt.rcParams['figure.autolayout'] = True

sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.2)

import plotly.io as pio
pio.templates.default = "plotly_white"

plt.rcParams['text.usetex'] = True


In [ ]:
# Color palettes

custom_palette = sns.color_palette('Set2')
sns.set_palette(custom_palette)

from cycler import cycler

# Define a custom color cycle
colors = ['#4E79A7', '#F28E2B', '#E15759', '#76B7B2', '#59A14F']
plt.rcParams['axes.prop_cycle'] = cycler(color=colors)


In [6]:
# Fonts
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
